## This script uses the PnF algorithm to run PnF tables for pairs of tickers in a list in order to construct a Relative Strength Matrix that ranks the ticker list from high to low relative strength.

In [ ]:
# Import credentials

import json
f = open("/. .<your file path here> . . /credentials.json")
credentials = json.load(f)

file_path = credentials['file_path']
intrinio_key = credentials['intrinio_key']


In [ ]:
# Import the usual Python libraries.

import pandas as pd
import numpy as np
from datetime import date, timedelta

# Import the Intrinio SDK for Python and set the API key.

import time
import intrinio_sdk as intrinio
from intrinio_sdk.rest import ApiException

intrinio.ApiClient().configuration.api_key['api_key'] = intrinio_key

# This is the path to where I store this notebook and downloaded files. You need to change this to the actual
# file directory you want to use.

global my_path
my_path = file_path


In [ ]:
# Configure the dates and variables for downloading close prices from Intrinio and running the PnF calcs.

tickerFile = 'rs_ticker_list.csv'  # Create a simple CSV file with two columns, ticker symbols and asset/company hames,
                                   # and enter the file name here.

# See the P&F article at https://school.stockcharts.com/doku.php?id=chart_analysis:pnf_charts:pnf_basics for an explanation
# of these next three parameters.
boxSize = .06
reversalThreshold = 3
reversalAmount = boxSize * reversalThreshold

historyDays = 180     # historyDays is the number of days going backward from today that we want to get close price data from Intrinio
                      # for, in order to capture enough price volatility to generate meaningful results.

lag = timedelta(days = 0)    # lag is the number of days prior to today to configure the end date if you want to see
                            # results as of some date in the past.
end_date = date.today() - lag

td = timedelta(days = historyDays)
start_date = end_date - td



In [ ]:
# Upload the ticker list here and set up the dataframes for plot symbol and PnF signal values.

def upload_ticker_list():
    
    global df_tickers
    global df_rs_plot
    global df_rs_signal

    df_tickers = pd.read_csv (my_path + tickerFile)
    no_tickers = len(df_tickers)

    # Set up the empty DF for plot symbols
    df_rs_plot = pd.DataFrame(columns=df_tickers['ticker'])
    df_rs_plot['ticker']=df_tickers.ticker
    df_rs_plot = df_rs_plot.rename_axis(None, axis=1).rename_axis('index', axis=0)
    df_rs_plot = df_rs_plot.set_index('ticker')
    
    # Set up the empty DF for signal values
    df_rs_signal = pd.DataFrame(columns=df_tickers['ticker'])
    df_rs_signal['ticker']=df_tickers.ticker
    df_rs_signal = df_rs_signal.rename_axis(None, axis=1).rename_axis('index', axis=0)
    df_rs_signal = df_rs_signal.set_index('ticker')
    
    return df_tickers, df_rs_plot, df_rs_signal


In [ ]:
# Function to get stock price history from Intrinio

def get_price_history(myTicker):
    
    global df_close_history
    
    frequency = 'daily'
    page_size = 10000
    next_page = ''

    response = intrinio.SecurityApi().get_security_stock_prices(identifier=myTicker, start_date=start_date, end_date=end_date, frequency=frequency, page_size=page_size, next_page=next_page)
    df_close_history = pd.DataFrame([x.to_dict() for x in response.stock_prices])[['date', 'adj_close']].sort_values('date')
    df_close_history.reset_index(drop = True, inplace = True)

    return df_close_history


In [ ]:
# Function to run the PnF calculations on the RS values calculated for each ticker pair.

def run_pnf_calcs():
    
    global data
    global plotSymbol
    global signalName
    
    data = df_relative_strength.copy()
    
    data['pctChange'] = data['rsValues'].pct_change()
    data['plotSymbol'] = np.nan
    data['reversal'] = 0
    data['signalName'] = np.nan
    data['lastHighPoint'] = np.nan
    data['lastLowPoint'] = np.nan

    data.loc[0, 'plotSymbol'] = 'X'
    data.loc[0, 'signalName'] = 'BUY'
    
    # Set all starting High Points equal to the close price on the first day of the time series.
    highPoint = data['rsValues'].iloc[0]
    lowPoint = data['rsValues'].iloc[0]
    lastHighPoint = data['rsValues'].iloc[0]
    lastLowPoint = data['rsValues'].iloc[0]

    # Start the loop on the second day, loop through each day's close price after that.
    for i in range(1, len(data)):
        
        if data['plotSymbol'].iloc[i - 1] == 'X':   #If previous Plot Symbol = "X", then:
        
            if data['rsValues'].iloc[i] >= data['rsValues'].iloc[i - 1]:     #If current price >= previous price, then:
                data.loc[i, 'plotSymbol'] = 'X'        # Today's Plot Symbol = "X".
                data.loc[i, 'signalName'] = data['signalName'].iloc[i - 1]

                if data['rsValues'].iloc[i] > highPoint:    #And if today's price is higher than the most recent high price, 
                    highPoint = data['rsValues'].iloc[i]       #then make today's price the  high price,
                    data.loc[i, 'signalName'] = data['signalName'].iloc[i - 1]   #and copy yesterday's signal to today.
            
                if data['rsValues'].iloc[i] > lastHighPoint:  #And if today's price is higher than the high point from the last X column,
                    data.loc[i, 'signalName'] = "BUY"           #then today's signal = "BUY".

                
            elif data['rsValues'].iloc[i] < highPoint * (1 - reversalAmount):     #Else if today's price is less than the previous high, times 1 - reversal,
                data.loc[i, 'plotSymbol'] = 'O'                                     #the Plot Symbol reverses "O",
                lowPoint = data['rsValues'].iloc[i]                                   #and the  low point is today's price,
                data.loc[i, 'reversal'] = 1                                         #and reversal = 1,
                lastHighPoint = highPoint                                               #and lastHighPoint = most recent high point
            
                if data['rsValues'].iloc[i] < lastLowPoint:   #And if today's price is lower than the low point from the last O column,
                    data.loc[i, 'signalName'] = "SELL"          #then today's signal = "SELL".
                else:
                    data.loc[i, 'signalName'] = data['signalName'].iloc[i - 1]   #Else copy yesterday's signal to today.
            
            else:
                data.loc[i, 'plotSymbol'] = 'X'  #Else, Plot Symbol = "X" (price is down but not enough to triger a reversal)
                data.loc[i, 'signalName'] = data['signalName'].iloc[i - 1]   #and copy yesterday's signal to today.
            

        if data['plotSymbol'].iloc[i - 1] == 'O':   #If previous Plot Symbol = "O", then:
        
            if data.loc[i, 'rsValues'] < data['rsValues'].iloc[i - 1]:            #If current price <= previous price, then:
                data.loc[i, 'plotSymbol'] = 'O'         # Today's Plot Symbol = "O".
                data.loc[i, 'signalName'] = data['signalName'].iloc[i - 1]
            
                if data['rsValues'].iloc[i] < lowPoint:       #And if today's price is lower than the most recent low price, 
                    lowPoint = data['rsValues'].iloc[i]         #then make today's price the  low price.
                    data.loc[i, 'signalName'] = data['signalName'].iloc[i - 1]   #and copy yesterday's signal to today.
                
                if data['rsValues'].iloc[i] < lastLowPoint:   #And if today's price is lower than the low point from the last O column,
                    data.loc[i, 'signalName'] = "SELL"         #then today's signal = "SELL".

                
            elif data['rsValues'].iloc[i] > lowPoint * (1 + reversalAmount):       #Else if today's price is greater than the previous high, times 1 + reversal,
                data.loc[i, 'plotSymbol'] = 'X'                                       #the Plot Symbol reverses to "X",
                highPoint = data['rsValues'].iloc[i]                                    #and the  high point is today's price,
                data.loc[i, 'reversal'] = 1                                           #and reversal = 1,
                lastLowPoint = lowPoint                                                   #and lastLowPoint = most recent low point
            
                if data['rsValues'].iloc[i] > lastHighPoint:  #And if today's price is higher than the high point from the last X column,
                    data.loc[i, 'signalName'] = "BUY"          #then today's signal = "BUY".
                else:
                    data.loc[i, 'signalName'] = data['signalName'].iloc[i - 1]     #Else copy yesterday's signal to today.
            
            else:
                data.loc[i, 'plotSymbol'] = 'O'  #Else, Plot Symbol = "O" (price is up but not enough to triger a reversal)
                data.loc[i, 'signalName'] = data['signalName'].iloc[i - 1]   #and copy yesterday's signal to today.
            
        data.loc[i, 'lastHighPoint'] = lastHighPoint  #lastHighPoint = current "lastHighPoint"
        data.loc[i, 'lastLowPoint'] = lastLowPoint    #lastLowPoint = current "lastLowPoint"
        
    return data
    

In [ ]:
# Get the historical price data for each ticker

def get_ticker_data(df_tickers):
    
    global df_close_histories

    df_close_histories = pd.DataFrame(columns = ['date', 'adj_close'])

    for index, x in df_tickers.iterrows():

        myTicker = x['ticker']
        get_price_history(myTicker)

        df_close_history['ticker'] = myTicker
        df_close_histories = pd.concat([df_close_histories, df_close_history], axis = 0)

    df_close_histories = df_close_histories.reset_index(drop = True)

    return df_close_histories


In [ ]:
# This is the part of the script that calculates the RS values and generates the matrix tables.

def run_the_rs_calcs(boxSize, reversalAmount, df_tickers, df_close_histories):
    
    global df_relative_strength

    df_tickers1 = df_tickers.copy()    #Start with a copy of the ticker list.

    for index, x in df_tickers1.iterrows():     #Iterate through the tickers in the list.

        myTicker = x['ticker']                  #Define the first ticker in the pair
        #get_price_history(myTicker)             #Get the history data from Intrinio (or from TMI data tables)

        df_prices1 = df_close_histories.loc[df_close_histories['ticker'] == myTicker][['date', 'adj_close']]
        df_prices1.rename(columns={'adj_close':'close1'}, inplace=True)   #Capture the adj close prices from the history dataframe

        df_tickers2 = df_tickers[df_tickers.ticker != myTicker].copy()    #Define the second ticker list, which a copy of the original
                                                                          #list without the first ticker in the current pair.    
        for index, x in df_tickers2.iterrows():   #Iterate through the second ticker list to get the second ticker for each pair.

            myTicker2 = x['ticker']              #Define the second ticker in the pair
            #get_price_history(myTicker2)         #Get the history data for the second ticker

            df_prices2 = df_close_histories.loc[df_close_histories['ticker'] == myTicker2][['date', 'adj_close']]
            df_prices2.rename(columns={'adj_close':'close2'}, inplace=True)   #Capture the adj close prices from the history dataframe

            df_relative_strength = pd.merge(df_prices1, df_prices2, on='date', how='left')    #Merge the two history dataframes to line up
                                                                                              #the close prices by date for each ticker in the pair.
            df_relative_strength['rsValues'] = df_relative_strength['close1']/df_relative_strength['close2']     #Calculate the relative strength values by dividing the first 
                                                                                                                 #ticker close price by the second ticker close price
            run_pnf_calcs()     #Run the PnF calculations on the calculated relative strength values

            #This line below is optional. It will export each RS value PnF table for QC inspection if desired.
            #data.to_csv(my_path + "RS_Calcs/rs_PnF_" + myTicker + "_" + myTicker2 + ".csv", index = True, header=True)

            df_rs_signal.loc[myTicker, myTicker2] = data['signalName'].iloc[len(data)-1]   #Capture the BUY/SELL signal from the last row of the PnF dataframe for the ticker
                                                                                           #and save it to the PnF signal dataframe
            df_rs_plot.loc[myTicker, myTicker2] = data['plotSymbol'].iloc[len(data)-1]     #Capture the X/O plot symbol from the last row of the PnF dataframe for the ticker
                                                                                           #and save it to the PnF plot symbol dataframe

    df_rs_signal['buyCount'] = df_rs_signal[df_rs_signal == "BUY"].count(1)     #Count up the number of BUY values in each row of the PnF signal dataframe
    df_rs_plot['xCount'] = df_rs_plot[df_rs_plot == "X"].count(1)               #Count up the number of X values in each row of the PnF plot symbol dataframe

    df_rs_signal.to_csv(my_path + "RS_Calcs/rs_signals.csv", index = True, header=True)  #Save the complete PnF signal dataframe to a CSV file
    df_rs_plot.to_csv(my_path + "RS_Calcs/rs_plots.csv", index = True, header=True)      #Save the complete PnF plot symbol dataframe to a CSV file
    
    print(df_rs_signal)
    print(df_rs_plot)
    
    # Merge the X count and BUY count columns to a separate dataframe, add up the values and sort by the total in descending order.
    df_rs_counts = pd.merge(df_tickers, df_rs_signal.buyCount, on='ticker', how='outer')
    df_rs_counts = pd.merge(df_rs_counts, df_rs_plot.xCount, on='ticker', how='outer')
    df_rs_counts['totalCount'] = df_rs_counts.sum(1)
    df_rs_counts = df_rs_counts.sort_values(['buyCount', 'xCount'], ascending = [False, False])
    
    return df_rs_counts



In [ ]:
# Run the RS calculation process.

upload_ticker_list()
get_ticker_data(df_tickers)
run_the_rs_calcs(boxSize, reversalAmount, df_tickers, df_close_histories)


In [ ]:
# Save the final dataframe to a CSV file if you want to.

df_rs_counts.to_csv(my_path + "RS_Calcs/rs_counts.csv", index = True, header=True)